In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import torch
import matplotlib.pyplot as plt
import pickle5 as pickle
import plotly.express as px
import itertools
import argparse
import scipy.stats
import scipy.special as special
from typing import Dict, List, Any, Tuple

In [2]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42)

In [3]:
def read_pickle(file_path: str) -> Any:
	with open(file_path, "rb") as handle:
		return pickle.load(handle)

In [4]:
def write_pickle(file: Any, file_path: str) -> None:
    with open(file_path, 'wb') as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
def get_word_freqs(i2s):
    in_v = Counter()
    out_v = Counter()
    
    for txt in i2s["In"]:
        tokens = txt.split()
        in_v.update(tokens)

    for txt in i2s["Out"]:
        tokens = txt.split()
        out_v.update(tokens)
    
    total = sum(in_v.values())
    for k in in_v:
        in_v[k] /= total

    total = sum(out_v.values())
    for k in out_v:
        out_v[k] /= total
        
    return in_v, out_v

In [6]:
def get_rarity(in_txt, out_txt, in_v, out_v):
    in_toks = in_txt.split()
    out_toks = out_txt.split()
    
    in_rarity, out_rarity = 0, 0
    in_len, out_len = len(in_toks), len(out_toks)
    
    for tok in in_toks:
        in_rarity += in_v[tok]
        
    in_rarity /= in_len
    
    for tok in out_toks:
        out_rarity += out_v[tok]
    
    out_rarity /= out_len
    
    return -np.log(in_rarity), -np.log(out_rarity)

In [7]:
STRING_TRUNCATE = 50

def get_scores(dir_path: str, converge_epoch: int, string_truncate: int) -> Tuple[Dict[int, Dict[str, List[float]]], Dict[str, List[Any]]]:
    file_list = os.listdir(dir_path)
    idx_to_sentences: Dict[int, Dict[str, str]] = read_pickle(os.path.join(dir_path, "idx_to_sentences.pickle"))

    file_list = [f for f in file_list if f[:5] == "epoch"]
    file_list = [f for f in file_list if int(f.split("_")[0].replace("epoch", "")) > 3 and int(f.split("_")[0].replace("epoch", "")) < converge_epoch]
    file_list = sorted(file_list, key= lambda s: int(s.split("_")[1].replace("stepidx", "")))

    # print("Loading files in:", dir_path)
    idxs, ppls, chias, bleus = [], [], [], []
    for file_name in file_list:
        file_path = f"{dir_path}/{file_name}"
        # print(file_name)
        if "ppl" in file_path:
            ppls.extend(read_pickle(file_path).tolist())
        elif "chia" in file_path:
            chias.extend(read_pickle(file_path).tolist())
        elif "bleu" in file_path:
            bleus.extend(read_pickle(file_path))
        elif "idx" in file_path:
            idxs.extend(read_pickle(file_path).tolist())
        else:
            output_csv_name = file_path

    items = list(zip(idxs, ppls, chias, bleus))
    items = sorted(items, key=lambda i: i[0])
    idx_dict: Dict[int, Dict[str, List[float]]] = {}
    for item in items:
        if item[0] not in idx_dict:
            idx_dict[item[0]] = {"inv_ppl": [1 / item[1]], "chia": [item[2]], "bleu": [item[3]]}
        else:
            idx_dict[item[0]]["inv_ppl"].append(1 / item[1])
            idx_dict[item[0]]["chia"].append(item[2])
            idx_dict[item[0]]["bleu"].append(item[3])

    i2s = {"Index": [], "In": [], "Out": [], "In Len": [], "Out Len": [], "In Rarity": [], "Out Rarity": []}

    for k, v in idx_to_sentences.items():
        i2s["Index"].append(k)
        i2s["In"].append(v["in"])
        i2s["Out"].append(v["out"])
        i2s["In Len"].append(len(v["in"].split()))
        i2s["Out Len"].append(len(v["out"].split()))

    in_v, out_v = get_word_freqs(i2s)
    for k, v in idx_to_sentences.items():
        in_rarity, out_rarity = get_rarity(v["in"], v["out"], in_v, out_v)
        i2s["In Rarity"].append(in_rarity)
        i2s["Out Rarity"].append(out_rarity)

    return idx_dict, i2s

In [8]:
from collections import Counter

def create_vocab(df):
	in_v = Counter()
	out_v = Counter()
    
	for idx, txt in df["In"].items():
		tokens = txt.split()
		in_v.update(tokens)
         
	for idx, txt in df["Out"].items():
		tokens = txt.split()
		out_v.update(tokens)

	return set(in_v.keys()), set(out_v.keys()), in_v, out_v

In [9]:
def calculate_statistics(epoch: int, idx_dict: Dict[int, Dict[str, List[float]]], i2s: Dict[str, List[Any]]) -> pd.DataFrame:
	idx_mean_var_dict: Dict[int, Dict[str, Tuple[float, float]]] = {}
	idx_mean_var_list: List[Tuple[int, float, float, float, float, float, float, float, float]] = []
	score_names = ["inv_ppl", "chia", "bleu"]
	for idx, scores in idx_dict.items():
		scores_list = []
		for score_name in score_names:
			score_arr = np.array(scores[score_name][:epoch])
			mean = score_arr.mean()
			var = score_arr.var()
			scores_list.extend([mean, var])
		
		idx_mean_var_list.append(tuple((idx, *scores_list)))

	i2s_df = pd.DataFrame.from_dict(i2s)


	df = pd.DataFrame(idx_mean_var_list, columns =['Index', 'Confidence - Inverse PPL', 'Variability - Inverse PPL', \
													'Confidence - CHIA', 'Variability - CHIA', \
													'Confidence - BLEU', 'Variability - BLEU'])

	cartography = pd.merge(df, i2s_df, on="Index")

	return cartography

In [10]:
def load_scores(dir_path: str, plot_path: str, converge_epoch: int) -> None:
	idx_dict = get_scores(dir_path, plot_path, converge_epoch)
	
	for epoch in trange(3, converge_epoch, 2):
		df = calculate_statistics(epoch, idx_dict)

		plot_types = ["inv_ppl", "chia", "bleu"]

		for plot_type in tqdm(plot_types, "Plots"):
			plot(df, plot_path, str(epoch), plot_type)

In [11]:
def save_subset(subset_df: pd.DataFrame, ds_name: str, subset_fname: str) -> None:
    subset_idx = subset_df["Index"].tolist()
    subset_idx = [int(i) for i in subset_idx]
    subset_idx = set(subset_idx)
    
    os.makedirs(os.path.join("subsets", ds_name), exist_ok=True)
    write_pickle(subset_idx, os.path.join("subsets", ds_name, subset_fname))
    print(f"subset_idx: {len(subset_idx)}")

In [12]:
from pprint import pprint

def choose_subset(df: pd.DataFrame, metric: str, criteria: str, ds_name: str, subset_fname:str, ratio:float = 0.33, write=True) -> pd.DataFrame:
    assert metric in ["Inverse PPL", "Neg PPL", "CHIA", "BLEU"]
    assert criteria in ["Easy to Learn", "Ambiguous", "Hard to Learn", "Random"]
    
    if criteria == "Easy to Learn":
        sort_by = f"Confidence - {metric}"
        ascending = False
    elif criteria == "Ambiguous":
        sort_by = f"Variability - {metric}"
        ascending = False
    elif criteria == "Hard to Learn":
        sort_by = f"Confidence - {metric}"
        ascending = True
        
    if criteria == "Random":
        sorted_df = df.sample(frac=1)
    else:
        sorted_df = df.sort_values(by=[sort_by], ascending=ascending)

    sorted_df = sorted_df.reset_index(drop=True)
    subset_df = sorted_df.iloc[:int(len(df)*ratio),:]
    
    all_in_v, all_out_v, _, _ = create_vocab(df)
    subset_in_v, subset_out_v, subset_in_v_counts, subset_out_v_counts = create_vocab(subset_df)

    add_ex_i = []
    remove_ex_i = []
    
    for i in trange(int(len(df)*ratio), len(df)):
        new_in, new_out = sorted_df.iloc[i, 7], sorted_df.iloc[i, 8]
        new_in_tokens, new_out_tokens = set(new_in.split()), set(new_out.split())
        
        if (new_in_tokens - subset_in_v) or (new_out_tokens - subset_out_v):
            # print(f"In vocab dif: {(new_in_tokens - subset_in_v)}")
            # print(f"Out vocab dif: {(new_out_tokens - subset_out_v)}")
            add_ex_i.append(i)
            subset_in_v = subset_in_v.union(new_in_tokens)
            subset_out_v = subset_out_v.union(new_out_tokens)
            subset_in_v_counts.update(new_in.split())
            subset_out_v_counts.update(new_out.split())
            
    in_counter = subset_in_v_counts
    out_counter = subset_out_v_counts
    
    removed_amount = 0
    for i in trange(int(len(df)*ratio)-1, -1, -1):
        if len(remove_ex_i) == len(add_ex_i):
            break
            
        ex_in, ex_out = sorted_df.iloc[i, 7], sorted_df.iloc[i, 8]
        ex_in_counter, ex_out_counter = Counter(ex_in.split()), Counter(ex_out.split())
        
        upd_in_counter = in_counter - ex_in_counter
        upd_out_counter = out_counter - ex_out_counter
        
        ex_in_words, ex_out_words = list(set(ex_in.split())), list(set(ex_out.split()))
        
        remove = True
        for word in ex_in_words:
            if upd_in_counter[word] <= 1:
                remove = False
        
        for word in ex_out_words:
            if upd_out_counter[word] <= 1:
                remove = False
                
        if remove:
            in_counter = upd_in_counter
            out_counter = upd_out_counter
            remove_ex_i.append(i)
    
    subset_df = pd.concat([subset_df, df.iloc[add_ex_i]])
    subset_df = subset_df.drop(remove_ex_i, axis=0)
    subset_df = subset_df.reset_index(drop=True)
    
    assert all_in_v == set(in_counter.keys()), "The process is wrong"
    assert all_out_v == set(out_counter.keys()), "The process is wrong 2"
    
    if write:
        save_subset(subset_df, ds_name, subset_fname)
    
    print(len(remove_ex_i), len(add_ex_i))
    
    return subset_df

In [13]:
def combine_subsets(df: pd.DataFrame, subset_dfs: List[pd.DataFrame], ds_name: str, subset_fname: str) -> pd.DataFrame:
    
    combined_set = pd.concat(subset_dfs)
    combined_set = combined_set.drop_duplicates(keep="first")
    
    if len(combined_set) > (len(df) / 2):
        combined_set = combined_set.iloc[:int(len(df) / 2)]
    else:
        count = 0
        while len(combined_set) < (len(df) / 2):
            example = df.sample(n=1)
            if not example.iloc[0]["In"] in combined_set['In'].tolist():
                combined_set = combined_set.append(example)
                
    save_subset(combined_set, ds_name, subset_fname)
    
    return combined_set

In [14]:
def plot(df, path_name, extra_path_info, plot_type="inv_ppl"):
	if plot_type == "inv_ppl":
		fig = px.scatter(df, x="Variability - Inverse PPL", y="Confidence - Inverse PPL", custom_data=['In abbv.', 'Out abbv.', 'In Len', 'Out Len'], color='Confidence - BLEU', range_color=[0,1])
		fig.update_layout(yaxis_range=[0, 1])
		fig.update_traces(
			hovertemplate="<br>".join([
				"Variability - Inverse PPL: %{x}",
				"Confidence - Inverse PPL: %{y}",
				"In: %{customdata[0]}",
				"Out: %{customdata[1]}",
                "In Len: %{customdata[2]}",
                "Out Len: %{customdata[3]}", 
			])
		)
	elif plot_type == "chia":
		fig = px.scatter(df, x="Variability - CHIA", y="Confidence - CHIA", custom_data=['In abbv.', 'Out abbv.', 'In Len', 'Out Len'], color='Confidence - BLEU', range_color=[0,1])
		fig.update_layout(yaxis_range=[0, 1])
		fig.update_traces(
			hovertemplate="<br>".join([
				"Variability - CHIA: %{x}",
				"Confidence - CHIA: %{y}",
				"In: %{customdata[0]}",
				"Out: %{customdata[1]}",
                "In Len: %{customdata[2]}",
                "Out Len: %{customdata[3]}", 
			])
		)
	elif plot_type == "bleu":
		fig = px.scatter(df, x="Variability - BLEU", y="Confidence - BLEU", custom_data=['In abbv.', 'Out abbv.', 'In Len', 'Out Len'], color='Confidence - BLEU', range_color=[0,1])
		fig.update_layout(yaxis_range=[0, 1])
		fig.update_traces(
			hovertemplate="<br>".join([
				"Variability - BLEU: %{x}",
				"Confidence - BLEU: %{y}",
				"In: %{customdata[0]}",
				"Out: %{customdata[1]}",
                "In Len: %{customdata[2]}",
                "Out Len: %{customdata[3]}", 
			])
		)	
	fig.update_traces(marker=dict(size=3), selector=dict(mode='markers'))
	fig.update_layout(
		autosize=False,
		width=800,
		height=900
	)
	fig.show()

In [15]:
STRING_TRUNCATE = 120

mtrc2abv = {"Inverse PPL": "inv_ppl", "Neg PPL": "neg_ppl", "CHIA": "chia", "BLEU": "bleu"}
crit2abv = {"Easy to Learn": "easy_to_learn", "Ambiguous": "ambiguous", "Hard to Learn": "hard_to_learn", "Random": "random"}
create_fname = lambda m, cr, c_e: f"{mtrc2abv[m]}_{crit2abv[cr]}_{c_e}.pickle"
create_ratio_fname = lambda m, cr, c_e, rto: f"{mtrc2abv[m]}_{crit2abv[cr]}_{c_e}_{rto}.pickle"
create_comb_fname = lambda m, cr1, cr2, c_e: f"{mtrc2abv[m]}_{crit2abv[cr1]}_{crit2abv[cr2]}_{c_e}.pickle"
outputs_path = lambda x: f"../scores/{x}"

In [16]:
# i2s = read_pickle(os.path.join(outputs_path("cfq"), "idx_to_sentences.pickle"))
# i2s_htl = read_pickle(os.path.join(outputs_path("cfq"), "idx_to_sentences_htl_20.pickle"))
# len(i2s), len(i2s_htl)

In [31]:
DATASET_NAME = "cogs"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Hard to Learn"
CONVERGE_EPOCH = 10

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/cogs


KeyboardInterrupt: 

In [ ]:
df.describe()

In [ ]:
subset_df.describe()

In [17]:
DATASET_NAME = "cfq"
OUTPUTS_PATH = outputs_path(DATASET_NAME)

METRICS = ["Inverse PPL", "CHIA"]
CRITERIA = ["Hard to Learn", "Easy to Learn", "Ambiguous",  "Random"]
COMBINED_CRITERIA = list(itertools.combinations(["Hard to Learn", "Ambiguous", "Easy to Learn"], 2))
RATIO = 0.5
CONVERGE_EPOCHS = [20]

for CONVERGE_EPOCH in CONVERGE_EPOCHS:
    for METRIC in METRICS:
        for CRITERION in CRITERIA:
            idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
            df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
            idx_fname = create_ratio_fname(METRIC, CRITERION, CONVERGE_EPOCH, RATIO)
            subset_df = choose_subset(df, METRIC, CRITERION, DATASET_NAME, idx_fname, ratio=RATIO)
            desc_df = subset_df.describe()
            print(f"{METRIC} - {CRITERION}: ", f'In Len: {desc_df["In Len"][1]:.2f}, Out Len: {desc_df["Out Len"][1]:.2f}, In Rarity: {desc_df["In Rarity"][1]:.2f}, Out Rarity: {desc_df["Out Rarity"][1]:.2f}')

  0%|          | 0/47871 [00:00<?, ?it/s]


subset_idx: 47871
0 0
Inverse PPL - Hard to Learn:  In Len: 15.21, Out Len: 31.00, In Rarity: 3.44, Out Rarity: 2.84


  0%|          | 9/47871 [00:00<00:08, 5744.75it/s]


subset_idx: 47867
9 9
Inverse PPL - Easy to Learn:  In Len: 11.86, Out Len: 24.49, In Rarity: 3.49, Out Rarity: 2.81


  0%|          | 7/47871 [00:00<00:08, 5430.02it/s]


...

subset_idx: 47869
9 9
CHIA - Easy to Learn:  In Len: 11.58, Out Len: 23.78, In Rarity: 3.50, Out Rarity: 2.81


  0%|          | 3/47871 [00:00<00:11, 4279.90it/s]


subset_idx: 47871
3 3
CHIA - Ambiguous:  In Len: 13.99, Out Len: 29.49, In Rarity: 3.45, Out Rarity: 2.85


  0%|          | 1/47871 [00:00<00:18, 2641.25it/s]


subset_idx: 47870
1 1
CHIA - Random:  In Len: 13.53, Out Len: 27.74, In Rarity: 3.47, Out Rarity: 2.83


In [22]:
DATASET_NAME = "cfq"
OUTPUTS_PATH = outputs_path(DATASET_NAME)

METRICS = ["Inverse PPL", "CHIA"]
CRITERIA = ["Hard to Learn", "Easy to Learn", "Ambiguous",  "Random"]
COMBINED_CRITERIA = list(itertools.combinations(["Hard to Learn", "Ambiguous", "Easy to Learn"], 2))
CONVERGE_EPOCH = 20

for METRIC in METRICS:
    for CRITERIA in COMBINED_CRITERIA:
        subset_dfs = []
        for CRITERION in CRITERIA:
            idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
            df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
            idx_fname = create_fname(METRIC, CRITERION, CONVERGE_EPOCH)
            subset_df = choose_subset(df, METRIC, CRITERION, DATASET_NAME, idx_fname, write=False)
            subset_dfs.append(subset_df)
        idx_fname = create_comb_fname(METRIC, CRITERIA[0], CRITERIA[1], CONVERGE_EPOCH)
        combined_set_df = combine_subsets(df, subset_dfs, DATASET_NAME, idx_fname)
        
        print(len(combined_set_df) / len(df))
        desc_df = subset_df.describe()
        #print(METRIC, CRITERION, f'In Len Mean: {desc_df["In Len"][1]}', f'Out Len Mean: {desc_df["Out Len"][1]}', f'In Rar Mean: {desc_df["In Rarity"][1]}', f'Out Rar Mean: {desc_df["Out Rarity"][1]}')

KeyboardInterrupt: 

In [96]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len,In Rarity,Out Rarity
count,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000,31595.000000
mean,47779.527583,0.468341,0.042448,0.708931,0.015143,0.598322,0.016847,16.092325,32.812629,3.430451,2.846160
std,27656.778684,0.070039,0.013744,0.047820,0.005074,0.045859,0.006459,4.110981,9.780604,0.176236,0.115194
min,2.000000,0.041971,0.000641,0.407099,0.001791,0.362612,0.001225,5.000000,11.000000,3.097469,2.566864
25%,23861.000000,0.429849,0.033184,0.681643,0.011528,0.570814,0.012113,13.000000,27.000000,3.300994,2.767585
50%,47720.000000,0.484256,0.041778,0.717357,0.014689,0.603874,0.016207,16.000000,31.000000,3.399570,2.843143
75%,71780.500000,0.523359,0.051236,0.742551,0.018286,0.630335,0.020929,19.000000,38.000000,3.521709,2.906923
max,95741.000000,0.555554,0.137260,0.845808,0.039034,0.743310,0.043566,29.000000,95.000000,4.686115,3.541374


In [97]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len,In Rarity,Out Rarity
count,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000,95743.000000
mean,47871.000000,0.597085,0.052975,0.782078,0.015463,0.669742,0.019771,13.534890,27.743208,3.465526,2.826699
std,27638.767749,0.111095,0.015563,0.065361,0.004721,0.066339,0.006750,4.614813,9.115588,0.223521,0.142301
min,0.000000,0.041971,0.000641,0.407099,0.001791,0.362612,0.001225,3.000000,11.000000,3.078264,2.566864
25%,23935.500000,0.524352,0.042325,0.741590,0.012121,0.627365,0.014906,10.000000,21.000000,3.308469,2.733988
50%,47871.000000,0.610798,0.052760,0.792068,0.015118,0.675890,0.019426,13.000000,26.000000,3.422888,2.811724
75%,71806.500000,0.681696,0.063200,0.831535,0.018423,0.716923,0.024243,17.000000,33.000000,3.572995,2.892543
max,95742.000000,0.890003,0.137260,0.937200,0.039919,0.856787,0.053432,29.000000,95.000000,4.781837,3.576896


In [94]:
DATASET_NAME = "scan_length"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Hard to Learn"
CONVERGE_EPOCH = 30

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

KeyboardInterrupt: 

In [ ]:
df.describe()

In [ ]:
subset_df.describe()

In [21]:
DATASET_NAME = "scan_jump"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Hard to Learn"
CONVERGE_EPOCH = 30

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/scan_jump


100%|██████████| 9829/9829 [00:00<00:00, 22867.38it/s]


In vocab dif: {'jump'}
Out vocab dif: {'I_JUMP'}


  0%|          | 1/4841 [00:00<00:01, 4009.85it/s]

1 1


In [22]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,14670.00000,14670.000000,1.467000e+04,14670.000000,1.467000e+04,14670.000000,14670.000000,14670.000000,14670.000000
mean,7334.50000,0.811877,3.109984e-02,0.870645,1.570789e-02,0.675831,0.015743,6.643763,12.727812
std,4235.00856,0.092899,2.224530e-02,0.069127,1.362532e-02,0.213221,0.011695,2.208942,9.162258
min,0.00000,0.276485,3.660198e-10,0.529350,3.663849e-10,0.131650,0.000000,1.000000,1.000000
25%,3667.25000,0.751254,1.256169e-02,0.823701,4.239430e-03,0.622686,0.006194,6.000000,6.000000
50%,7334.50000,0.798249,2.819625e-02,0.861600,1.211250e-02,0.717946,0.013982,7.000000,10.000000
75%,11001.75000,0.869623,4.748547e-02,0.920733,2.486033e-02,0.829987,0.023542,8.000000,18.000000
max,14669.00000,0.999992,1.394370e-01,0.999992,7.669706e-02,0.969229,0.068510,9.000000,48.000000


In [23]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000
mean,7356.418715,0.720289,0.052241,0.797500,0.029197,0.632905,0.025675,6.817187,8.457963
std,4260.666859,0.048495,0.017266,0.034889,0.011621,0.091986,0.010002,1.349258,3.931534
min,1.000000,0.276485,0.007619,0.529350,0.002780,0.249460,0.002889,1.000000,1.000000
25%,3697.000000,0.698229,0.039868,0.781987,0.020214,0.576173,0.018181,6.000000,5.000000
50%,7295.000000,0.730197,0.052269,0.806718,0.028243,0.650905,0.024752,7.000000,8.000000
75%,10996.000000,0.753725,0.063736,0.823363,0.037331,0.698421,0.032411,8.000000,11.000000
max,14666.000000,0.812255,0.139437,0.835363,0.076697,0.856253,0.068510,9.000000,24.000000


In [24]:
import gc
gc.collect()

69

In [25]:
DATASET_NAME = "pcfg"
OUTPUTS_PATH = outputs_path(DATASET_NAME)
METRIC = "CHIA"
CRITERIA = "Hard to Learn"
CONVERGE_EPOCH = 140

idx_dict, i2s = get_scores(OUTPUTS_PATH, CONVERGE_EPOCH, STRING_TRUNCATE)
df = calculate_statistics(CONVERGE_EPOCH, idx_dict, i2s)
idx_fname = create_fname(METRIC, CRITERIA, CONVERGE_EPOCH)
subset_df = choose_subset(df, METRIC, CRITERIA, DATASET_NAME, idx_fname)

Loading files in: ../scores/pcfg


  0%|          | 0/27115 [00:00<?, ?it/s]


0 0


In [26]:
df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000,82168.000000
mean,41083.500000,0.657339,0.126839,0.712272,0.103254,0.506744,0.061316,17.699883,8.895884
std,23720.002797,0.124807,0.033832,0.102243,0.033217,0.068689,0.033513,9.410670,7.794699
min,0.000000,0.096914,0.003238,0.186722,0.001352,0.175278,0.000825,3.000000,2.000000
25%,20541.750000,0.597530,0.105946,0.658073,0.079857,0.472124,0.031961,11.000000,4.000000
50%,41083.500000,0.680250,0.131590,0.726545,0.104896,0.520684,0.060412,16.000000,7.000000
75%,61625.250000,0.743193,0.154147,0.784396,0.130419,0.556863,0.088102,23.000000,11.000000
max,82167.000000,0.981210,0.194359,0.984010,0.176240,0.754878,0.149322,71.000000,736.000000


In [27]:
subset_df.describe()

,Index,Confidence - Inverse PPL,Variability - Inverse PPL,Confidence - CHIA,Variability - CHIA,Confidence - BLEU,Variability - BLEU,In Len,Out Len
count,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000,27115.000000
mean,41163.802508,0.517232,0.139458,0.597169,0.127739,0.501446,0.090461,24.506362,15.096810
std,23767.251788,0.103914,0.033883,0.079702,0.025657,0.069216,0.025137,9.282645,10.553437
min,14.000000,0.096914,0.005126,0.186722,0.014304,0.175278,0.003750,4.000000,2.000000
25%,20653.000000,0.464303,0.120784,0.561201,0.113865,0.466188,0.075694,18.000000,10.000000
50%,41280.000000,0.548299,0.149316,0.621488,0.133957,0.520229,0.094642,23.000000,13.000000
75%,61746.500000,0.596771,0.165431,0.657101,0.146735,0.551279,0.108756,30.000000,18.000000
max,82167.000000,0.658802,0.194359,0.683924,0.176240,0.660227,0.149322,71.000000,736.000000


subset_df_in = set(subset_df["In"].tolist())
subset_df_out = set(subset_df["Out"].tolist())

subset_pkl = read_pickle("../scores/cogs/idx_to_sentences.pickle")

subset_pkl_in = []
subset_pkl_out = []

for i, text in subset_pkl.items():
    subset_pkl_in.append(text["in"])
    subset_pkl_out.append(text["out"])

subset_pkl_in = set(subset_pkl_in)
subset_pkl_out = set(subset_pkl_out)

subset_df_in - subset_pkl_in, len(subset_df_in), len(subset_pkl_in)